# Segmentation and Feature Extraction_2

## Overview
This notebook performs segmentation (nucleus and whole cell) and extracts features from the preprocessed images (Hybrid PMD → CLAHE, size-standardized) in `Dataset/Preprocessed Dataset`.

It computes:
- Morphology (area, perimeter, major/minor axis, circularity, eccentricity, solidity, aspect ratio) for nucleus and cell
- Color statistics and histograms in RGB, HSV, Lab
- Stain deconvolution (Hematoxylin/Eosin) means and stds
- Texture features: GLCM (contrast, correlation, energy, homogeneity) and LBP histogram

Outputs (CSV + masks/overlays) are saved to `Segmentation and Feature Extraction Results` inside the `Project-Phoenix` folder.

Run cells from top to bottom. If paths differ on your machine, change the `DATASET_DIR` variable in the setup cell.

## Setup and dependencies
If any import fails, the next cell will attempt to install the missing packages automatically. Adjust paths in the Config cell if needed.

In [2]:
# Imports and auto-install
import sys, subprocess, importlib

def ensure(pkg, import_name=None):
    import_name = import_name or pkg
    try:
        return importlib.import_module(import_name)
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        return importlib.import_module(import_name)

np = ensure('numpy')
pd = ensure('pandas')
cv2 = ensure('opencv-python', 'cv2')
skimage_measure = ensure('scikit-image', 'skimage.measure')
skimage_morph = ensure('scikit-image', 'skimage.morphology')
skimage_filters = ensure('scikit-image', 'skimage.filters')
skimage_color = ensure('scikit-image', 'skimage.color')
skimage_feature = ensure('scikit-image', 'skimage.feature')
skimage_exposure = ensure('scikit-image', 'skimage.exposure')
skimage_seg = ensure('scikit-image', 'skimage.segmentation')
from skimage.measure import regionprops_table, label
from skimage.morphology import remove_small_objects, disk, closing
from skimage.filters import threshold_otsu, threshold_sauvola, gaussian
from skimage.color import rgb2hsv, rgb2lab
# In skimage >= 0.25, use graycomatrix/graycoprops from skimage.feature.texture
from skimage.feature.texture import graycomatrix, graycoprops, local_binary_pattern
from skimage.exposure import rescale_intensity
from skimage.segmentation import clear_border
from pathlib import Path
from tqdm import tqdm
import json
import math
import os

print("Libraries ready.")

Libraries ready.


In [3]:
# Config
PROJECT_ROOT = Path(r"c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix")
DATASET_DIR = PROJECT_ROOT / "Dataset" / "Preprocessed Dataset"
RESULTS_DIR = PROJECT_ROOT / "Segmentation and Feature Extraction Results"
MASKS_DIR = RESULTS_DIR / "masks"
OVERLAYS_DIR = RESULTS_DIR / "overlays"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MASKS_DIR.mkdir(parents=True, exist_ok=True)
OVERLAYS_DIR.mkdir(parents=True, exist_ok=True)

# Image and feature params
GLCM_DISTANCES = [1, 2, 4]
GLCM_ANGLES = [0, np.pi/4, np.pi/2, 3*np.pi/4]
LBP_P = 8
LBP_R = 1
LBP_METHOD = 'uniform'

def list_images(root: Path):
    # Recursively list common image types
    exts = {".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp"}
    return [p for p in root.rglob('*') if p.suffix.lower() in exts]

print(f"Dataset: {DATASET_DIR}")
print(f"Saving results to: {RESULTS_DIR}")

Dataset: c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset
Saving results to: c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Segmentation and Feature Extraction Results


In [4]:
# Helper functions

def read_image(path: Path):
    img_bgr = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise ValueError(f"Failed to read: {path}")
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb


def simple_cell_segmentation(img_rgb):
    # Assumes preprocessed (denoised, CLAHE). Segment probable cell via thresholding on saturation/brightness.
    hsv = rgb2hsv(img_rgb)
    v = (hsv[..., 2] * 255).astype(np.uint8)
    v_blur = cv2.GaussianBlur(v, (5, 5), 0)
    thr = threshold_otsu(v_blur)
    cell_mask = (v_blur > thr).astype(np.uint8)
    cell_mask = closing(cell_mask, disk(3))
    cell_mask = remove_small_objects(cell_mask.astype(bool), 200).astype(np.uint8)
    cell_mask = clear_border(cell_mask).astype(np.uint8)
    return cell_mask


def simple_nucleus_segmentation(img_rgb, cell_mask):
    # Enhance nuclei via hematoxylin-like channel proxy: use Lab b/a and HSV S as cues
    lab = rgb2lab(img_rgb)
    hsv = rgb2hsv(img_rgb)
    # Combine channels: darker + higher saturation
    a = lab[..., 1]
    b = lab[..., 2]
    s = hsv[..., 1]
    proxy = rescale_intensity((s - (a + b)/200.0), in_range='image', out_range=(0, 1))
    proxy_u8 = (proxy * 255).astype(np.uint8)
    proxy_u8 = cv2.GaussianBlur(proxy_u8, (3, 3), 0)
    thr = threshold_otsu(proxy_u8[cell_mask.astype(bool)]) if cell_mask.sum()>0 else threshold_otsu(proxy_u8)
    nuc_mask = (proxy_u8 > thr).astype(np.uint8)
    nuc_mask = nuc_mask * cell_mask  # nucleus inside cell
    nuc_mask = closing(nuc_mask, disk(2))
    nuc_mask = remove_small_objects(nuc_mask.astype(bool), 50).astype(np.uint8)
    return nuc_mask


def largest_region(mask):
    lab = label(mask)
    if lab.max() == 0:
        return mask
    # Select largest component
    areas = np.bincount(lab.ravel())
    areas[0] = 0
    idx = areas.argmax()
    return (lab == idx).astype(np.uint8)


def contour_perimeter(mask):
    cnts, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts:
        return 0.0
    return float(cv2.arcLength(max(cnts, key=cv2.contourArea), True))


def morph_props(mask, tag):
    lab = label(mask)
    props = regionprops_table(lab, properties=(
        'area', 'label', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'solidity'
    ))
    df = pd.DataFrame(props)
    if df.empty:
        return {f'{tag}_area': 0, f'{tag}_perimeter': 0, f'{tag}_major_axis': 0, f'{tag}_minor_axis': 0,
                f'{tag}_circularity': 0, f'{tag}_eccentricity': 0, f'{tag}_solidity': 0, f'{tag}_aspect_ratio': 0}
    # Use largest region
    i = int(df['area'].idxmax())
    area = float(df.loc[i, 'area'])
    major = float(df.loc[i, 'major_axis_length'])
    minor = float(df.loc[i, 'minor_axis_length'])
    eccentricity = float(df.loc[i, 'eccentricity'])
    solidity = float(df.loc[i, 'solidity'])
    perim = contour_perimeter((lab == df.loc[i, 'label']).astype(np.uint8))
    circularity = (4 * math.pi * area / (perim ** 2)) if perim > 0 else 0.0
    aspect_ratio = (major / minor) if minor > 0 else 0.0
    return {
        f'{tag}_area': area,
        f'{tag}_perimeter': perim,
        f'{tag}_major_axis': major,
        f'{tag}_minor_axis': minor,
        f'{tag}_circularity': circularity,
        f'{tag}_eccentricity': eccentricity,
        f'{tag}_solidity': solidity,
        f'{tag}_aspect_ratio': aspect_ratio,
    }


def color_stats(img_rgb, mask=None, prefix='rgb'):
    if mask is not None:
        idx = mask.astype(bool)
    else:
        idx = slice(None)
    ch_means = img_rgb[idx].reshape(-1, 3).mean(axis=0)
    ch_stds = img_rgb[idx].reshape(-1, 3).std(axis=0)
    d = {f'{prefix}_mean_R': float(ch_means[0]), f'{prefix}_mean_G': float(ch_means[1]), f'{prefix}_mean_B': float(ch_means[2]),
         f'{prefix}_std_R': float(ch_stds[0]), f'{prefix}_std_G': float(ch_stds[1]), f'{prefix}_std_B': float(ch_stds[2])}
    return d


def color_histograms(img_rgb, mask=None, prefix='rgb', bins=32):
    hist = {}
    for i, ch in enumerate('RGB'):
        h = cv2.calcHist([img_rgb], [i], mask, [bins], [0, 256]).flatten()
        h = (h / (h.sum() + 1e-8)).astype(float)
        # store a compact summary: first 8 bins and entropy
        entropy = float(-(h * np.log(h + 1e-12)).sum())
        for k in range(8):
            hist[f'{prefix}_hist_{ch}_b{k}'] = float(h[k])
        hist[f'{prefix}_hist_{ch}_entropy'] = entropy
    return hist


def convert_and_stats(img_rgb, mask=None):
    hsv = (rgb2hsv(img_rgb) * 255).astype(np.uint8)
    lab = rgb2lab(img_rgb)
    lab_u8 = np.zeros_like(img_rgb, dtype=np.uint8)
    lab_u8[..., 0] = np.clip((lab[..., 0] / 100) * 255, 0, 255)
    lab_u8[..., 1] = np.clip((lab[..., 1] + 128), 0, 255)
    lab_u8[..., 2] = np.clip((lab[..., 2] + 128), 0, 255)
    out = {}
    out.update(color_stats(img_rgb, mask, 'rgb'))
    out.update(color_stats(hsv, mask, 'hsv'))
    out.update(color_stats(lab_u8, mask, 'lab'))
    out.update(color_histograms(img_rgb, mask, 'rgb'))
    out.update(color_histograms(hsv, mask, 'hsv'))
    out.update(color_histograms(lab_u8, mask, 'lab'))
    return out


def stain_deconvolution_stats(img_rgb, mask=None):
    # Simple H&E proxy using Macenko-like log-optical density without full deconvolution to keep it simple
    img = img_rgb.astype(np.float32) + 1.0
    od = -np.log(img / 255.0)
    # Project onto two fixed stain vectors (approximate H and E)
    H = np.array([0.65, 0.70, 0.29], dtype=np.float32)
    E = np.array([0.07, 0.99, 0.11], dtype=np.float32)
    H = H / np.linalg.norm(H)
    E = E / np.linalg.norm(E)
    Hc = od @ H
    Ec = od @ E
    if mask is not None:
        idx = mask.astype(bool)
        Hc = Hc[idx]
        Ec = Ec[idx]
    return {
        'stain_H_mean': float(np.mean(Hc)), 'stain_H_std': float(np.std(Hc)),
        'stain_E_mean': float(np.mean(Ec)), 'stain_E_std': float(np.std(Ec)),
    }


def glcm_features(gray_u8, mask=None, prefix='glcm'):
    if mask is not None:
        # mask region; fill outside with mode to not bias co-occurrence severely
        m = mask.astype(bool)
        if m.sum() == 0:
            roi = gray_u8
        else:
            roi = gray_u8.copy()
            mode_val = np.bincount(roi[m].ravel()).argmax()
            roi[~m] = mode_val
    else:
        roi = gray_u8
    glcm = greycomatrix(roi, distances=GLCM_DISTANCES, angles=GLCM_ANGLES, levels=256, symmetric=True, normed=True)
    feats = {}
    for prop in ['contrast', 'correlation', 'energy', 'homogeneity']:
        vals = greycoprops(glcm, prop)
        feats[f'{prefix}_{prop}_mean'] = float(np.mean(vals))
        feats[f'{prefix}_{prop}_std'] = float(np.std(vals))
    return feats


def lbp_features(gray_u8, mask=None, prefix='lbp'):
    lbp = local_binary_pattern(gray_u8, LBP_P, LBP_R, method=LBP_METHOD)
    lbp_u8 = np.uint8(255 * (lbp - lbp.min()) / (lbp.max() - lbp.min() + 1e-8))
    if mask is not None and mask.sum() > 0:
        vals = lbp_u8[mask.astype(bool)].ravel()
    else:
        vals = lbp_u8.ravel()
    hist, _ = np.histogram(vals, bins=32, range=(0, 256), density=True)
    d = {f'{prefix}_b{k}': float(hist[k]) for k in range(len(hist))}
    d[f'{prefix}_entropy'] = float(-(hist * np.log(hist + 1e-12)).sum())
    return d


def make_overlay(img_rgb, mask, color=(255, 0, 0), alpha=0.35):
    overlay = img_rgb.copy()
    col = np.array(color, dtype=np.uint8)
    idx = mask.astype(bool)
    overlay[idx] = (alpha * col + (1 - alpha) * overlay[idx]).astype(np.uint8)
    return overlay

print("Helper functions ready.")

Helper functions ready.


In [5]:
# Process dataset
records = []
images = list_images(DATASET_DIR)
print(f"Found {len(images)} images.")

for img_path in tqdm(images):
    rel = img_path.relative_to(DATASET_DIR)
    out_prefix = RESULTS_DIR / rel.with_suffix('')
    out_prefix.parent.mkdir(parents=True, exist_ok=True)

    try:
        img = read_image(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Segment cell and nucleus
        cell_mask = simple_cell_segmentation(img)
        cell_mask = largest_region(cell_mask)
        nuc_mask = simple_nucleus_segmentation(img, cell_mask)
        nuc_mask = largest_region(nuc_mask)

        # Save masks and overlays
        cell_mask_path = MASKS_DIR / rel.with_suffix('').as_posix().replace('/', '__')
        nuc_mask_path = MASKS_DIR / (rel.with_suffix('').as_posix().replace('/', '__') + "_nuc")
        overlay_cell = make_overlay(img, cell_mask, (255, 0, 0))
        overlay_nuc = make_overlay(img, nuc_mask, (0, 255, 0))
        cv2.imwrite(str((Path(str(cell_mask_path)) ).with_suffix('.png')), cell_mask*255)
        cv2.imwrite(str((Path(str(nuc_mask_path)) ).with_suffix('.png')), nuc_mask*255)
        cv2.imwrite(str((OVERLAYS_DIR / (rel.with_suffix('').as_posix().replace('/', '__') + "_cell_overlay.png"))), cv2.cvtColor(overlay_cell, cv2.COLOR_RGB2BGR))
        cv2.imwrite(str((OVERLAYS_DIR / (rel.with_suffix('').as_posix().replace('/', '__') + "_nuc_overlay.png"))), cv2.cvtColor(overlay_nuc, cv2.COLOR_RGB2BGR))

        # Morphology
        feats = {}
        feats.update(morph_props(cell_mask, 'cell'))
        feats.update(morph_props(nuc_mask, 'nucleus'))

        # Color/staining stats
        feats.update(convert_and_stats(img, cell_mask))
        feats.update({f'nuc_{k}': v for k, v in convert_and_stats(img, nuc_mask).items()})
        feats.update(stain_deconvolution_stats(img, cell_mask))
        feats.update({f'nuc_{k}': v for k, v in stain_deconvolution_stats(img, nuc_mask).items()})

        # Texture features
        feats.update(glcm_features(gray, cell_mask, 'glcm_cell'))
        feats.update(glcm_features(gray, nuc_mask, 'glcm_nuc'))
        feats.update(lbp_features(gray, cell_mask, 'lbp_cell'))
        feats.update(lbp_features(gray, nuc_mask, 'lbp_nuc'))

        feats['image_rel_path'] = str(rel).replace('\\', '/')
        records.append(feats)

    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        continue

# Save CSV
csv_path = RESULTS_DIR / 'features.csv'
pd.DataFrame(records).to_csv(csv_path, index=False)
print(f"Saved features: {csv_path}")

Found 8098 images.


  0%|          | 0/8098 [00:00<?, ?it/s]C:\Users\meetb\AppData\Local\Temp\ipykernel_53104\2748481638.py:97: RuntimeWarning: Mean of empty slice.
  ch_means = img_rgb[idx].reshape(-1, 3).mean(axis=0)
C:\Users\meetb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\meetb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\meetb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users

Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\001_cyt_cyt01.png: name 'greycomatrix' is not defined
Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\001_cyt_cyt02.png: name 'greycomatrix' is not defined
Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\001_cyt_cyt03.png: name 'greycomatrix' is not defined


  0%|          | 5/8098 [00:00<15:39,  8.61it/s]

Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\001_cyt_cyt04.png: name 'greycomatrix' is not defined
Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\001_cyt_cyt05.png: name 'greycomatrix' is not defined


...

 16%|█▌        | 1270/8098 [01:58<10:40, 10.66it/s]

Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\164_cyt_cyt14.png: name 'greycomatrix' is not defined
Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\164_cyt_cyt15.png: name 'greycomatrix' is not defined
Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\164_cyt_cyt16.png: name 'greycomatrix' is not defined


 16%|█▌        | 1272/8098 [01:58<10:37, 10.71it/s]

Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\164_cyt_cyt17.png: name 'greycomatrix' is not defined
Error processing c:\Meet\Projects\Project_8_Phoenix_Cervical Cancer Image Classification\Project-Phoenix\Dataset\Preprocessed Dataset\im_Dyskeratotic\164_cyt_cyt18.png: name 'greycomatrix' is not defined


KeyboardInterrupt: 